In [38]:


import tensorflow as tf
print(tf.__version__)

2.15.0


In [39]:
import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import Model, Sequential

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError

from tensorflow.keras.layers import Dense, Conv1D, LSTM, Lambda, Reshape, RNN, LSTMCell

import warnings
warnings.filterwarnings('ignore')

In [40]:
plt.rcParams['figure.figsize'] = (10, 7.5)
plt.rcParams['axes.grid'] = False

In [41]:
tf.random.set_seed(42)
np.random.seed(42)

In [42]:
train_df = pd.read_csv('data/train.csv', index_col='datetime',parse_dates=True)
val_df = pd.read_csv('data/val.csv', index_col='datetime',parse_dates=True)
test_df = pd.read_csv('data/test.csv', index_col='datetime',parse_dates=True)

print(train_df.shape, val_df.shape, test_df.shape)


(2716, 26) (776, 26) (389, 26)


In [43]:
#DataWindow Class

In [44]:
class DataWindow():
    def __init__(self, input_width, label_width, shift,
                 train_df=train_df, val_df=val_df, test_df=test_df,
                 label_columns=None):

        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
        self.column_indices = {name: i for i, name in enumerate(train_df.columns)}

        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def split_to_inputs_labels(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:,:,self.column_indices[name]] for name in self.label_columns],
                axis=-1
            )
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels


    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=32
        )

        ds = ds.map(self.split_to_inputs_labels)
        return ds

    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)

    @property
    def sample_batch(self):
        result = getattr(self, '_sample_batch', None)
        if result is None:
            result = next(iter(self.train))
            self._sample_batch = result
        return result

In [45]:
class Baseline(Model):
    def __init__(self, label_index=None):
        super().__init__()
        self.label_index = label_index

    def call(self, inputs):
        if self.label_index is None:
            return inputs

        elif isinstance(self.label_index, list):
            tensors = []
            for index in self.label_index:
                result = inputs[:, :, index]
                result = result[:, :, tf.newaxis]
                tensors.append(result)
            return tf.concat(tensors, axis=-1)

        result = inputs[:, :, self.label_index]
        return result[:,:,tf.newaxis]

Multi-output baseline model


In [46]:
mo_single_step_window = DataWindow(input_width=1, label_width=1, shift=1, label_columns=['precip','rain_sum','river_discharge'])
mo_wide_window = DataWindow(input_width=14, label_width=14, shift=1, label_columns=['precip','rain_sum','river_discharge'])

In [47]:
column_indices = {name: i for i, name in enumerate(train_df.columns)}

In [48]:
print(column_indices['precip'])
print(column_indices['rain_sum'])
print(column_indices['river_discharge'])


25
24
23


In [49]:
mo_baseline_last = Baseline(label_index=[23,24,25])

mo_baseline_last.compile(loss=MeanSquaredError(), metrics=[MeanAbsoluteError()])

mo_val_performance = {}
mo_performance = {}

mo_val_performance['Baseline - Last'] = mo_baseline_last.evaluate(mo_wide_window.val)
mo_performance['Baseline - Last'] = mo_baseline_last.evaluate(mo_wide_window.test, verbose=0)

24/24 [==============================] - 0s 2ms/step - loss: 0.0075 - mean_absolute_error: 0.0510


In [50]:
print(mo_performance['Baseline - Last'][1])

0.038097064942121506


Implementing a deep neural network as a multi-output model

In [51]:
def compile_and_fit(model, window, patience=3, max_epochs=50):
    early_stopping = EarlyStopping(monitor='val_loss',
                                   patience=patience,
                                   mode='min')

    model.compile(loss=MeanSquaredError(),
                  optimizer=Adam(),
                  metrics=[MeanAbsoluteError()])

    history = model.fit(window.train,
                       epochs=max_epochs,
                       validation_data=window.val,
                       callbacks=[early_stopping])

    return history

In [52]:
mo_dense = Sequential([
    Dense(units=64, activation='relu'),
    Dense(units=64, activation='relu'),
    Dense(units=3)
])

history = compile_and_fit(mo_dense, mo_single_step_window)

mo_val_performance['Dense'] = mo_dense.evaluate(mo_single_step_window.val)
mo_performance['Dense'] = mo_dense.evaluate(mo_single_step_window.test, verbose=0)

Epoch 1/50
85/85 [==============================] - 1s 4ms/step - loss: 0.0055 - mean_absolute_error: 0.0459 - val_loss: 0.0027 - val_mean_absolute_error: 0.0333
Epoch 2/50
85/85 [==============================] - 0s 3ms/step - loss: 0.0043 - mean_absolute_error: 0.0380 - val_loss: 0.0027 - val_mean_absolute_error: 0.0360
Epoch 3/50
85/85 [==============================] - 0s 3ms/step - loss: 0.0038 - mean_absolute_error: 0.0344 - val_loss: 0.0023 - val_mean_absolute_error: 0.0279
Epoch 4/50
85/85 [==============================] - 0s 3ms/step - loss: 0.0038 - mean_absolute_error: 0.0342 - val_loss: 0.0023 - val_mean_absolute_error: 0.0279
Epoch 5/50
85/85 [==============================] - 0s 3ms/step - loss: 0.0037 - mean_absolute_error: 0.0338 - val_loss: 0.0024 - val_mean_absolute_error: 0.0323
Epoch 6/50
25/25 [==============================] - 0s 2ms/step - loss: 0.0024 - mean_absolute_error: 0.0276


**LSTM**

In [53]:
mo_lstm_model = Sequential([
    LSTM(32, return_sequences=True),
    Dense(units = 3)
])

history = compile_and_fit(mo_lstm_model, mo_wide_window)

mo_val_performance = {}
mo_performance = {}

mo_val_performance['LSTM'] = mo_lstm_model.evaluate(mo_wide_window.val)
mo_performance['LSTM'] = mo_lstm_model.evaluate(mo_wide_window.test, verbose=0)

Epoch 1/50
85/85 [==============================] - 3s 12ms/step - loss: 0.0115 - mean_absolute_error: 0.0700 - val_loss: 0.0035 - val_mean_absolute_error: 0.0404
Epoch 2/50
85/85 [==============================] - 1s 7ms/step - loss: 0.0049 - mean_absolute_error: 0.0431 - val_loss: 0.0030 - val_mean_absolute_error: 0.0364
Epoch 3/50
85/85 [==============================] - 1s 6ms/step - loss: 0.0044 - mean_absolute_error: 0.0394 - val_loss: 0.0027 - val_mean_absolute_error: 0.0336
Epoch 4/50
85/85 [==============================] - 1s 7ms/step - loss: 0.0041 - mean_absolute_error: 0.0369 - val_loss: 0.0026 - val_mean_absolute_error: 0.0329
Epoch 5/50
85/85 [==============================] - 1s 7ms/step - loss: 0.0039 - mean_absolute_error: 0.0353 - val_loss: 0.0025 - val_mean_absolute_error: 0.0313
Epoch 6/50
85/85 [==============================] - 1s 7ms/step - loss: 0.0037 - mean_absolute_error: 0.0339 - val_loss: 0.0025 - val_mean_absolute_error: 0.0314
Epoch 7/50
85/85 [=========

In [55]:
predicted_results = mo_baseline_last.predict(mo_wide_window.test)
predicted_array= predicted_results[0]

my_array = np.array(predicted_array)

df_raw = pd.DataFrame(my_array)

df = df_raw.rename(columns={0: "river_discharge", 1: "rain_sum",2:"precip"})


df.head(14)

12/12 [==============================] - 0s 1ms/step


,river_discharge,rain_sum,precip
0,0.034817,0.095918,0.060897
1,0.094042,0.269388,0.087821
2,0.111271,0.186735,0.232692
3,0.156856,0.185714,0.278846
4,0.230079,0.530612,0.250000
5,0.275664,0.551020,0.071795
6,0.203877,0.318367,0.069872
7,0.127782,0.066327,0.012179
8,0.073941,0.078571,0.040385
9,0.053841,0.011224,0.058333


In [56]:
mo_lstm_model.save("lstm_dhaka_model.h5")